```
@作者: 阿凯爱玩机器人
@QQ: 244561792
@微信: xingshunkai
@邮箱: xingshunkai@qq.com
@网址: deepsenserobot.com
@B站: "阿凯爱玩机器人"
```

## 导入依赖

In [1]:
import numpy as np
import open3d as o3d
# 自定义库
from kyle_robot_toolbox.open3d import *

pybullet build time: Jun  3 2022 02:05:55


## 载入点云

In [2]:
# 载入场景点云
scene_pcd = o3d.io.read_point_cloud(f"./data/example/box_pose/pcd_rect_roi.pcd")

# 载入盒子表面点云
box_panel_pcd = o3d.io.read_point_cloud("./data/example/box_pose/box_panel_pcd.pcd")

# 赋值为蓝色
box_panel_pcd.paint_uniform_color([0.0, 0.0, 1.0])

# 点云可视化
o3d.visualization.draw_geometries([scene_pcd], window_name="盒子场景点云")

In [3]:
# 创建彩色相机坐标系的Mesh
cam_corrd_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.05)
# 显示盒子上表面点云与相机坐标系
o3d.visualization.draw_geometries([box_panel_pcd,  cam_corrd_mesh], \
                                  window_name="盒子上表面点云与相机坐标系")

## 获取点云中心点

In [4]:
center_point = box_panel_pcd.get_center()
print(f"盒子点云中心点: {center_point}")

盒子点云中心点: [0.048 0.002 0.224]


In [5]:
# 可视化，在质心位置绘制小球
mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.002)
mesh_sphere.compute_vertex_normals()
# 小球平移
mesh_sphere.translate(center_point.reshape(-1), relative=False)
# 给小球上色为红色
mesh_sphere.paint_uniform_color([0.0, 1.0, 0])
# 创建可视化窗口
draw_geometry([scene_pcd, mesh_sphere],\
                window_name="绘制盒子上表面中心点")

## 盒子高度测量

之前有做过工作台平面拟合，有了盒子上表面的质心坐标就可以计算盒子的高度。 

In [6]:
# 载入工作台的平面数据
desktop_panel_model = np.loadtxt("./data/example/box_pose/desktop_panel_model.txt", delimiter=",")

[A, B, C, D] = desktop_panel_model
print(f"拟合桌面平面的表达式: {A:.2f}x + {B:.2f}y + {C:.2f}z + {D:.2f} = 0")

拟合桌面平面的表达式: 0.05x + 0.01y + 1.00z + -0.26 = 0


In [7]:
# 计算中心点距离平面的距离
distance = get_distance_to_panel(A, B, C, D, center_point)
print(f"盒子高度(点到平面距离) {distance:.3f}, 单位m")

盒子高度(点到平面距离) 0.029, 单位m
